In [1]:
import os
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import scipy.sparse as sp
from keras import Model
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
def _load_data_x_y(x_path: str,
                   y_path: str,
                   do_label_flip: bool
                   ) -> tf.data.Dataset:
    format = x_path.split('.')[-2:]
    if format[-1] == 'npy':
        x = np.load(x_path)
    elif format[-1] == 'npz' and format[-2] == 'csr':
        x = sp.load_npz(x_path).toarray()
    else:
        raise ValueError(f'Unknown format: {format}')

    y = np.load(y_path)
    y_max = np.max(y)
    y_min = np.min(y)

    if do_label_flip:
        y = y_max - y + y_min

    y = y.astype(np.float32) / 4.0

    ret = tf.data.Dataset.from_tensor_slices((x, y))

    return ret

In [3]:
tf.keras.backend.clear_session()

In [4]:
init_model_path = './sentiment140_init_model.h5'
x_train_path = './sentiment140_x_train.csr.npz'
y_train_path = './sentiment140_y_train.npy'
x_test_path = './sentiment140_x_test.csr.npz'
y_test_path = './sentiment140_y_test.npy'
batch_size = 1024
shuffle_train = True
repeat_train = True

In [5]:
model: Model = tf.keras.models.load_model(init_model_path, compile=True)
model.summary()

Metal device set to: Apple M1


2022-03-20 19:41:49.472062: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-20 19:41:49.472174: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sentiment140_init_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 60)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 60, 301)      8153187     ['input_1[0][0]']                
                                                                                                  
 bidirectional (Bidirectional)  (None, 60, 256)      440320      ['embedding[0][0]']              
                                                                                                  
 multi_head_attention (MultiHea  (None, 60, 256)     821856      ['bidirectional[0][0]',          
 dAttention)                                                      'bidirecti

In [6]:
with tf.device('/cpu:0'):
    # train_ds = _load_data_x_y(x_train_path, y_train_path, do_label_flip=False)
    # test_ds = _load_data_x_y(x_test_path, y_test_path, do_label_flip=False)
    # # val_ds = None
    # # TODO: validation dataset may NOT be `None`

    all_ds = _load_data_x_y(x_train_path, y_train_path, do_label_flip=False)
    test_ds = all_ds.take(len(all_ds) // 5)
    train_ds = all_ds.skip(len(all_ds) // 5)

    steps_per_epoch = (len(train_ds) + batch_size - 1) // batch_size

    if shuffle_train:
        train_ds = train_ds.shuffle(buffer_size=10000)

    if repeat_train:
        train_ds = train_ds.repeat()

    train_ds = train_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    test_ds = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [7]:
%%time
history = model.fit(
    train_ds,
    epochs=3,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_ds,
    validation_freq=1,
)

Epoch 1/3


2022-03-20 19:41:52.349512: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-20 19:41:52.359674: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-20 19:41:53.066450: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-20 19:41:53.203134: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-20 19:41:56.727176: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-20 19:41:56.902092: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


  14/1250 [..............................] - ETA: 4:30:04 - loss: 0.5907 - binary_accuracy: 0.6966

KeyboardInterrupt: 